<h1>Gaussian Process Autoencoder</h1>
Python 3.7.2<br>
Pytorch 1.0.1<br>
matplotlib 3.0.2<br>
jupyter 1.0.0<br>
jupyter lab 0.35.3<br></h4>

In [1]:
"Jupyter Lab Extension"
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

"패키지 불러오기"
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F

import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

print("CUDA Available :")
print(torch.cuda.is_available())

CUDA Available :
True


In [2]:
data = load_iris().data
label = load_iris().target
print(data.shape, label.shape)

(150, 4) (150,)


In [3]:
pca = PCA(n_components=2).fit(data)
data_new = pca.transform(data)
data_new.shape

(150, 2)

In [4]:
plt.figure()
plt.scatter(data_new[:, 0], data_new[:, 1], c=label)
plt.show()

FigureCanvasNbAgg()

In [120]:
class GPAE(nn.Module):
    def __init__(self, ndim_x, ndim_v, twin_kernel = False):
        super(GPAE, self).__init__()
        "입력변수 및 잠재변수 차원 지정"
        self.ndim_x = ndim_x
        self.ndim_v = ndim_v
        
        "GP 모델에 적합시킬 데이터 저장"
        self.data_x = torch.zeros(0, self.ndim_x)
        self.data_v = torch.zeros(0, 0)
        self.recon = torch.zeros(0, self.ndim_x)
        
        "저장한 데이터에서 설명변수들을 가지고 L2 pairwise distance(Euclidean distance)를 구할 것"
        self.dist_x = torch.zeros((self.ndim_x, 0, 0))
        
        "Latent dimension의 gram matrix 저장"
        self.gram_v = torch.zeros((self.ndim_v, 0, 0), requires_grad = True)
        
        "Kernel Hyperparameter"
        self.KHP = torch.randn((self.ndim_x, self.ndim_v), requires_grad = True)
        self.KHP_decode = torch.randn((self.ndim_v, self.ndim_x), requires_grad = True)
        
        "Noise Parameter"
        self.beta_v = torch.randn(self.ndim_v, requires_grad = True)
        
        "weight, noise 의 효과를 양수로 유지"
        self.normalize = nn.Softplus()
        
        
    
    "Distance Matrix 계산 함수"
    def distance(self, data_input):
        N = data_input.shape[0]
        D = data_input.shape[1]
        dist = torch.zeros(D, N, N)
        for dim in range(D):
            term1 = (data_input[:, dim] ** 2).expand((N,N))
            term2 = data_input[:, dim].unsqueeze(0).transpose(0, 1) * data_input[:, dim].unsqueeze(0)
            dist[dim] = (term1 + term1.transpose(0,1) - 2 * term2).unsqueeze(0)
        return dist
    
    
    "새로운 데이터의 X값들 추가"
    def fit_x(self, x_new):
        self.data_x = torch.cat((self.data_x, x_new), 0)
        self.dist_x = self.distance(self.data_x)
    
    def noise(self):
        return torch.einsum("abc,a->abc", (torch.eye(self.data_x.shape[0]).unsqueeze(0).expand(self.ndim_v, self.data_x.shape[0], self.data_x.shape[0]), self.normalize(self.beta_v)))
        
    def encode(self):
        x = torch.einsum("abc,ad->dbc", (self.dist_x, self.normalize(self.KHP)))
        x = torch.exp(-x/2)
        x = x + self.noise()
        self.gram_v = x
        return (x.narrow(1,0,1) ** 2 + x.narrow(2,0,1) ** 2 - x)/2
    
    def decode(self):
        x = self.gram_v - self.noise()
        x = -2 * torch.log(torch.clamp(x, min = 1e-8))
        x = torch.einsum("abc,ad->dbc", (x, self.normalize(self.KHP_decode)))
        return x
        
        
    def forward(self):
        x = self.encode()        
        decomp = x.squeeze(0).svd()
        self.data_v = torch.mm(decomp[0].narrow(1, 0, 2), torch.diag(torch.sqrt(decomp[1].narrow(0, 0, 2))))
        return torch.log(torch.clamp(torch.det(self.gram_v.squeeze(0)), min=1e-6)) + torch.trace(self.data_v.transpose(0,1)@self.gram_v.squeeze(0)@self.data_v)

In [121]:
"모델 선언"
model = GPAE(4, 1)
model.fit_x(torch.from_numpy(data).type(torch.float32))

In [122]:
model.KHP

tensor([[ 1.8322],
        [-0.1370],
        [-0.7827],
        [ 0.2890]], requires_grad=True)

In [123]:
epoch = 300

lr = 1e-1
optimizer = torch.optim.Adam([model.KHP, model.KHP_decode, model.beta_v], lr = lr)

print(model.forward().detach())
for iteration in range(epoch):
    optimizer.zero_grad()
    loss = model.forward()
    loss.backward()
    optimizer.step()
print(model.forward().detach())

tensor(3017.4863)
tensor(268.7152)


In [124]:
data_new = model.data_v.detach().numpy()

In [125]:
plt.figure()
plt.scatter(data_new[:, 0], data_new[:, 1], c=label)
plt.show()

FigureCanvasNbAgg()

In [363]:
model.KHP

tensor([[14.1793, 16.6445],
        [16.9868, 20.1551],
        [12.1660, 16.6504],
        [17.1085, 18.4652]], requires_grad=True)

In [364]:
model.KHP_decode

tensor([[-0.4583,  0.3218, -0.3131, -0.8776],
        [ 1.6973, -1.1141,  0.7879, -3.9338]], requires_grad=True)

In [365]:
model.beta_v

tensor([-4.6001, -4.4250], requires_grad=True)